# Assignment 6 - Boosting

## Before you begin

Remember to:

1. Make your own copy of the notebook by pressing the "Copy to drive" button.
2. Expend all cells by pressing **Ctrl+[**

### Your IDs

✍️ Fill in your IDs in the cell below:

In [ ]:
## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
## Fill in your IDs (as a string)
student1_id = '...'
student2_id = '...'
## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

print('Hello ' + student1_id + ' & ' + student2_id)

### Importing Packages

In [ ]:
import numpy as np  # Numerical package (mainly multi-dimensional arrays and linear algebra)
import pandas as pd  # A package for working with data frames
import matplotlib.pyplot as plt  # A plotting package
import tqdm.notebook as tqdm  # A progress bar package

## Setup matplotlib to output figures into the notebook
%matplotlib inline

## Set some default values of the the matplotlib plots
plt.rcParams['figure.figsize'] = (6.0, 6.0)  # Set default plot's sizes
plt.rcParams['figure.dpi'] = 120  # Set default plot's dpi (increase fonts' size)
plt.rcParams['axes.grid'] = True  # Show grid by default in figures

## Labeled Voices Dataset

In this assignment we will use the same dataset as we have used in tutorial 11 (SVM) which contains the properties of 3168 voice samples along with the gender of the speaker.This dataset was collected and processed by Kory Becker and was published on [her website](http://www.primaryobjects.com/2016/06/22/identifying-the-gender-of-a-voice-using-machine-learning/)

According to the project's web page, the 3168 voice samples and their label were collected from the following resources:

- [The Harvard-Haskins Database of Regularly-Timed Speech](http://www.nsi.edu/~ani/download.html)
- Telecommunications & Signal Processing Laboratory (TSP) Speech Database at McGill University
- [VoxForge Speech Corpus](http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/8kHz_16bit/)
- [Festvox CMU_ARCTIC Speech Database at Carnegie Mellon University](http://festvox.org/cmu_arctic/)

Each voice track was then processed using a tool called [WarbleR](https://cran.r-project.org/web/packages/warbleR/warbleR.pdf) in order to extract 20 numerical features for each track.

The dataset can be found [here](https://technion046195.netlify.app/datasets/voice.csv)

### Loading the dataset

In [ ]:
data_file = 'https://technion046195.netlify.app/datasets/voice.csv'

## Loading the data
dataset = pd.read_csv(data_file)

dataset

## The Data Fields and Types

The list and descriptions of the data fields as copied from the project's website:

- **meanfreq**: mean frequency (in kHz)
- **sd**: standard deviation of frequency
- **median**: median frequency (in kHz)
- **Q25**: first quantile (in kHz)
- **Q75**: third quantile (in kHz)
- **IQR**: interquantile range (in kHz)
- **skew**: skewness (see note in specprop description)
- **kurt**: kurtosis (see note in specprop description)
- **sp.ent**: spectral entropy
- **sfm**: spectral flatness
- **mode**: mode frequency
- **centroid**: frequency centroid (see specprop)
- **meanfun**: average of fundamental frequency measured across acoustic signal
- **minfun**: minimum fundamental frequency measured across acoustic signal
- **maxfun**: maximum fundamental frequency measured across acoustic signal
- **meandom**: average of dominant frequency measured across acoustic signal
- **mindom**: minimum of dominant frequency measured across acoustic signal
- **maxdom**: maximum of dominant frequency measured across acoustic signal
- **dfrange**: range of dominant frequency measured across acoustic signal
- **modindx**: modulation index. Calculated as the accumulated absolute difference between

- **label**: The label of each track: male/female

We shall store all the relevant fields in a list named "x_fields" and the field of the label as "y_field"

In [ ]:
y_field = 'label'

x_fields = ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
            'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
            'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx']

## Train-Validation-Test split

✍️ Complete the code below to split the data into 60% train 20% validation set set and 20% test set similar to the last assignment

In [ ]:
n_samples = len(dataset)  # The total number of samples in the dataset

## Generate a random generator with a fixed seed
rand_gen = np.random.RandomState(1)

## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
## Generating a shuffled vector of indices
...

## Split the indices into 60% train / 20% validation / 20% test
...
train_indices = ...
val_indices = ...
test_indices = ...
## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

## Extract the sub datasets from the full dataset using the calculated indices
train_set = dataset.iloc[train_indices]
val_set = dataset.iloc[val_indices]
test_set = dataset.iloc[test_indices]

## AdaBoost

We will implement the AdaBoost algorithm using stumps (depth 1 trees). 

### An efficient calculation of the weighted misclassification rate.

In every step of AdaBoost we need to go over all the possibles stumps, going all the possible thresholds for each of the fields. It turns out that there is a quick way to calculate the weighted misclassification rate for all the thresholds for some fields at once. To simplify the derivation we will assume that each value of $\text{x}_k$ in the dataset is unique):

1. We will choose a specific field of $\mathbf{x}$: $\text{x}_k$.
2. We will sort the samples according to $x_k^{(i)}$.
3. We would like to calculate the weighted misclassification rate for stumps in the following form for each sample in the dataset $\boldsymbol{x}^{(m)}$:

$$
h(\boldsymbol{x})=2I\{x_k>x_k^{(m)}\}-1
=\begin{cases}
1 & x_k>x_k^{(m)} \\
-1 & \text{else} \\
\end{cases}
$$

4. After sorting the samples we can write the weighted misclassification rate for the above stump in the following way:

$$
\begin{aligned}
\sum_{i=1}^N w_iI\{y^{(i)}\neq h(\boldsymbol{x}^{(i)})\}
&=\sum_{i=1}^N w_iI\{y^{(i)}\neq(2I\{x_k^{(i)}>x_k^{(m)}\}-1)\}\\
&=\sum_{i=1}^N w_i\left( I\{y^{(i)}=1\}I\{x_k^{(i)}\leq x_k^{(m)}\}
                        +I\{y^{(i)}=-1\}I\{x_k^{(i)}>x_k^{(m)}\}\right)\\
&=\sum_{i=1}^N w_i\left( \tfrac{1}{2}(1+y^{(i)})I\{x_k^{(i)}\leq x_k^{(m)}\}
                        +\tfrac{1}{2}(1-y^{(i)})I\{x_k^{(i)}>x_k^{(m)}\}\right)\\
&=\tfrac{1}{2}\sum_{i=1}^N w_i\left( I\{x_k^{(i)}\leq x_k^{(m)}\}
                                    +I\{x_k^{(i)}>x_k^{(m)}\}
                                    +y^{(i)}\left( I\{x_k^{(i)}\leq x_k^{(m)}\}
                                                  -I\{x_k^{(i)}>x_k^{(m)}\}\right)
                              \right)\\
&=\tfrac{1}{2}\sum_{i=1}^N w_i\left(1+y^{(i)}\left(2I\{x_k^{(i)}\leq x_k^{(m)}\}-1\right)\right)\\
&=\tfrac{1}{2}\left( \sum_{i=1}^N w_i
                    -\sum_{i=1}^N w_i y^{(i)}
                    +2\sum_{i=1}^N w_i y^{(i)}I\{x_k^{(i)}\leq x_k^{(m)}\}
              \right)\\
&=\tfrac{1}{2}\left(1-\sum_{i=1}^N w_i y^{(i)}+2\sum_{i=1}^m w_i y^{(i)}\right)\\
\end{aligned}
$$

In the above formula the only dependency on $m$ is in the last summation. We can quickly calculate the values for all values of $m\in[1,N]$ using the function [numpy.cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html).

The method *find_threshold* in the code bellow implements this calculation.

### Implementation

✍️ Complete the code below to implement a the AdaBoost calculation:

In [ ]:
def stump_predict(dataset, field, threshold, sign):
    """
    An auxiliary function to calculate a stump classification, i.e.:
      h(x) = 2I{x > threshold} - 1

    Parameters
    ----------
    dataset: Pandas DataFrame
        The dataset on which to apply the classification.
    field: string
        The field on which to apply the threshold
    threshold: float
        The threshold
    sign: integer: -1 or 1
        An integer indication if the thresholf should be:
        - x > threshold (for sign = 1)
        - x < threshold (for sign = -1)        
    
    Return
    ------
    y_hat: ndarray
        The vector of predictions
    """

    if sign > 0:
        y_hat = (dataset[field].values > threshold) * 2 - 1
    else:
        y_hat = (dataset[field].values < threshold) * 2 - 1
    return y_hat
    
class AdaBoost:
    def __init__(self, train_set, x_fields, y_field):
        """
        Initizaling the class fields.
        
        Parameters
        ----------
        train_set: Pandas DataFrame
            The train set.
        x_fields: list of strings
            The fields of x in the dataset.
        y_field: string
            The field containting the label y.
        """

        self.train_set = train_set
        self.x_fields = x_fields
        self.y_field = y_field
        
        self.y_train = (self.train_set[self.y_field] == 'male').values * 2 - 1  # Extract the y vector as a -1,1 labels
        self.thresholds = []  # This field will store the stumps thresholds
        self.signs = []  # This field will store if the threshold should be > or <.
        self.fields = []  # This field will store the stump according to which the threshold will be applied.
        self.alphas = []  # This fields will store AbaBoost's coefficients.
        self.weights = np.ones(len(train_set)) / len(train_set)  # Initialized AdaBoost's wieghts.

    def find_threshold(self, field, sign, weights):
        """
        Given a field and a weight vector, this method finds the threshold which minimizes 
        the weighted misclassification rate.
        
        Parameters
        ----------
        field: string
            The field of x which is to be used for the thresholding.
        sign: integer: -1 or 1
            An integer indication if the thresholf should be:
            - x > threshold (for sign = 1)
            - x < threshold (for sign = -1)        
        weights: ndarray
            The 1D array of the AdaBoost weights of the samples.
        
        Returns
        -------
        threshold: float
            The optimal threshold.
        sign: integer: -1 or 1
            An integer indication if the thresholf should be:
            - x > threshold (for sign = 1)
            - x < threshold (for sign = -1)        
        """

        x = self.train_set[field].values  # Extract the relevant field.
        indices = np.argsort(x)  # Find the indices which sorts the samples.
        if sign < 0:
            indices = indices[::-1]  # if sign = -1 reverse indices so that x will be in descending order.

        sorted_x = x[indices]  # Sort x
        signed_sorted_weights = weights[indices] * self.y_train[indices]  # Calculate w_i * y_i.
        weighted_misclass_list = 0.5 * (1 - signed_sorted_weights.sum() + 2 * np.cumsum(signed_sorted_weights))  # Calculate the weighted misclassification rate for each threshold.
        
        index = np.argmin(weighted_misclass_list)  # Find the index which minimizes the weighted miscalssifiaction rate
        threshold = sorted_x[index]
        
        return threshold
        
    def step(self):
        """
        Preform a single AdaBoost step claculating the following parameters for the current step:
        - field: The field on which the stump will apply the thershold.
        - threshold: The stumps threshold.
        - alpha: The AdaBoost's coefficients for the current stump.
        """

        weights = self.weights.copy()  # Make a copy of the weights vector.
        
        ## Loop over all possible thresholds
        ## ---------------------------------
        ## Loop over all fields and over the sign, check the weighted misclassification rate, select the
        ## one with the minimal and store it's parameters (field, threshold and sign)
        best_field = None
        best_threshold = None
        best_sign = None
        best_weighted_misclass = np.inf

        for field in self.x_fields:
            for sign in [-1, 1]:
                threshold = self.find_threshold(field, sign, weights)
                
                ## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
                ## Calcualte the weighted misclassification rate: sum_over_i(w_i * I{y_i == h(x_i)})
                ## - Use the function "stump_predict" to calculate h(x_i)
                ## - Calcualte the expression on "self.train_set".
                ## - Use "self.y_train" for y_i
                weighted_misclass = ...
                ## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

                ## Check if the weighted misclassification rate is the best so far and store the relevant parameters
                if weighted_misclass < best_weighted_misclass:
                    best_field = field
                    best_threshold = threshold
                    best_sign = sign
                    best_weighted_misclass = weighted_misclass

        field = best_field
        threshold = best_threshold
        sign = best_sign
        weighted_misclass = best_weighted_misclass
        
        ## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
        ## Perform the AdaBoost step
        ## -------------------------
        ## Store the stump parameters
        self.fields.append(field)
        self.thresholds.append(...
        self.signs.append(...

        ## Calcualte alpha for the current stump
        alpha = ...
        self.alphas.append(alpha)

        ## Update the weights
        ## Calcualte the unnormalized weights
        weights *= ...
        ## Normalized the weights
        weights /= ...
        self.weights = weights
        ## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%
        
    def calc_objective(self):
        """
        An auxilizary method for calculating AdaBoost's objective.
        This function is not needed for running and using AdaBoost. It is used here only for ploting purpose.
        
        Returns
        -------
        objective: float
            AdaBoost's objective.
        """

        objective = 0
        for field, threshold, sign, alpha in zip(self.fields, self.thresholds, self.signs, self.alphas):
            objective += alpha * stump_predict(self.train_set, field, threshold, sign) * self.y_train
        objective = np.exp(-objective).mean()
        return objective
        
    def predict(self, dataset):
        """
        This method calculates the prediction using the set of stumps and alphas for a given dataset.
        
        Parameters
        ----------
        dataset: Pandas DataFrame
            The dataset on which to apply the classification.

        Return
        ------
        y_hat: ndarray
            The vector of predictions
        """

        raw_y_hat = pd.Series(0, index=dataset.index)  # Initializing the linear combination of predictions
        
        ## Loop over the stumps to calculate the linear combination of predictions sum_t(alpha_t * h_t(x))
        ## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
        for field, threshold, sign, alpha in zip(self.fields, self.thresholds, self.signs, self.alphas):
            raw_y_hat += ...
        ## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

        y_hat = np.sign(raw_y_hat).astype(int)  # Apply the sign function
        return y_hat

adaboost = AdaBoost(train_set, x_fields, y_field)  ## Create an AdaBoost object
adaboost.step()  # Preform a step
## Print the selected stump in the first step:
stump_sign = '>' if (adaboost.signs[0] > 0) else '<'
print(f'The selected stump: {adaboost.alphas[0]:.4f} * ({adaboost.fields[0]} {stump_sign} {adaboost.thresholds[0]:.2f})')

You should get some thing like: "1.5245 * (meanfun < 0.14)"

The following function calculates the misclassification rate for an AdaBoost instance on a given dataset.

In [ ]:
def calc_score(adaboost, dataset):
    """
    Calculates the misclassification rate of a predictor on a given dataset.

    Parameters
    ----------
    adaboost: AdaBoost
        An AdaBoost object
    dataset: Pandas DataFrame
        The dataset on which to apply the classification.
    
    Returns
    -------
    score: scalar
        The evaluated score on the dataset.
    """

    ## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
    y_hat = ...
    score = ...
    ## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%
    return score

print(f'The score on the train set is: {calc_score(adaboost, train_set):.4f}')

The score for a AdaBoost with a single step on the train set should be less then 5%

### Run AdaBoost

The following code runs AdaBoost for 500 steps

In [ ]:
adaboost = AdaBoost(train_set, x_fields, y_field)

steps_list = []
train_score_list = []
val_score_list = []
objective_list = []
weighted_misclass_list = []

for step in tqdm.tqdm(range(1, 501)):
    weighted_misclass = adaboost.step()

    if (step % 10) == 0:
        train_score = calc_score(adaboost, train_set)
        val_score = calc_score(adaboost, val_set)

        steps_list.append(step)
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        weighted_misclass_list.append(weighted_misclass)
        objective_list.append(adaboost.calc_objective())
    
## Plot
fig, ax = plt.subplots(figsize=(5, 5))

ax.plot(steps_list, train_score_list, label='Train')
ax.plot(steps_list, val_score_list, label='Validation')
ax.plot(steps_list, objective_list, label='Objective')
# ax.set_title(r'$\eta={' + f'{eta:g}' + r'}$')
ax.set_ylim(0, 0.05)
ax.set_xlabel('Step')
ax.set_ylabel('Score')
ax.legend();

Print the first 10 stump and coefficients

In [ ]:
for i in range(10):
    stump_sign = '>' if (adaboost.signs[i] < 0) else '<'
    print(f'Step {i+1}: {adaboost.alphas[i]:.4f} * ({adaboost.fields[i]} {stump_sign} {adaboost.thresholds[i]:.2f})')

## Final evaluation

Evaluate the model using the test set

✍️ Complete the code below the evaluate the misclassification rate on the test set

In [ ]:
## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
test_score = ...
## %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

print(f'The misclassification rate on the test is: {test_score}')

You should get less then 2.5% error (take a look at the results from tutorial 11).

## AdaBoost in scikit-learn

The SciKit-learn package also has an implementation for [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html).

The code bellow uses SciKit-learn to run AdaBoost for 500 steps.

✍️ Complete the code below the evaluate the misclassification rate on the test set (don't use the "calc_score" function).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=500, algorithm='SAMME')
clf.fit(train_set[x_fields].values, train_set[y_field].values)

## %%%%%%%%%%%%%%% Your code here - Begin %%%%%%%%%%%%%%%
test_score = ...
# %%%%%%%%%%%%%%% Your code here - End %%%%%%%%%%%%%%%%%

print(test_score)

## Submission

To submit your code download it as a **ipynb** file from Colab, and upload it to the course's website (Moodle). You can download this code by selecting **Download .ipynb** from the **file** menu.